In [1069]:
import numpy as np
import math
import matplotlib.pyplot as plt
import scipy.linalg as linalg
from numba import jit
import scipy

In [2]:
from scipy.stats import norm
def Black_Scholes_Call(S, K, r, vol, tau):
    """ 
    Black Scholes Model for European Call
    """
    d1 = (np.log(S / K) + (r + ((vol**2)/2.)*tau)) / (vol*np.sqrt(tau))
    d2 = d1 - vol*np.sqrt(tau)
    V = S * norm.cdf(d1) - np.exp(-r*tau) * K * norm.cdf(d2)
    
    return V

For this equation, we conclude that
\begin{align}
    \vec{a_{-1}} &= \frac{1}{2}\sigma^{2}\frac{\Delta\tau}{\Delta X^{2}} - \frac{\Delta\tau}{2\Delta X}(r-\frac{1}{2}\sigma^{2}) \\
    \vec{a_{0}} &=  1- \sigma^{2}\frac{\Delta\tau}{\Delta X^{2}} - r\Delta\tau\\
    \vec{a_{+1}} &=  \frac{\Delta\tau}{2\Delta X}(r-\frac{1}{2}\sigma^{2})+\frac{1}{2}\sigma^{2}\frac{\Delta\tau}{\Delta X^{2}}
\end{align}

In [1070]:
def FD(S, K, r, sigma, T, N, M, scheme="FCTS"):
    # Compute delta T
    dt = T/N
        
    s_max = 10000
    s_min = 1
    X = np.linspace(np.log(s_min), np.log(s_max), M)
    dx = (X[-1] - X[0])/(M-1);
    
    # Generate stock prices on log scale
    # Generate T * S grid
    V = np.zeros((M,N))

    # Put the discounted values into the first column
    V[:, 0] = [max(np.exp(s)-K,0) for s in X]
    V[0, -1] = s_max
    V[0, 0] = s_min
    # Fucking constants
    ss = sigma * sigma
    dxx = dx*dx
    
    # a-1, a0, a+1
    if scheme=="FCTS":
        f = (0.5*ss * dt / dxx)
        g = (r-0.5*ss) *dt / (2*dx)
        h = r * dt
        
        ad = f-g
        a0 = 1-2*f - h
        au = f+g
    else:       
        f = ((r-0.5*ss) * (dt/(4*dx)))
        g = (ss*dt)/(4*dxx)
        h = (r*dt/2)
        
        ad = g-f
        a0 = 1-2*g-h
        au = g+f
        
        
        bd = f-g
        b0 = 1+2*g+h
        bu = -f-g
        
        
    # Generate matrix A
    A = np.zeros((M,M))
    
    np.fill_diagonal(A[1:], ad)
    np.fill_diagonal(A[:,1:], au)
    np.fill_diagonal(A, a0)
    A[0,0], A[0,1], A[-1,-2], A[-1,-1] = 1,0, 0,1
    
    if scheme == "CN":
        B = np.zeros((M,M))
    
        np.fill_diagonal(B[1:], bd)
        np.fill_diagonal(B[:,1:], bu)
        np.fill_diagonal(B, b0)
        
        B[0,0], B[0,1], B[-1,-2], B[-1,-1] = 1,0, 0,1

    if scheme == "CN":
        for i in range(1,N):
            b = np.matmul(A, V[:, i-1])
            V[:, i] = scipy.sparse.linalg.spsolve(B, b)
    else:
        for i in range(1,N):
            V[:, i] = np.matmul(A, V[:, i-1])

    return A, V , X

def get_ans(result):
    nearest_idx = np.where(abs(np.exp(result[2])-S)==abs(np.exp(result[2])-S).min())[0]
    return result[1][nearest_idx, -1]

In [1060]:
sigma = 0.3
K = 110
S = 100
r = 0.04
T = 1
N = 1500 # time points
M =  500# space points

In [1061]:
Black_Scholes_Call(S, K, r, sigma, T)

9.625357828843697

In [1066]:
result = FD(S, K, r, sigma, T, N, M, scheme="FCTS")
get_ans(result)

array([9.18017364])

In [1064]:
result[2][abs(np.exp(result[2])-S)==abs(np.exp(result[2])-S).min()]

nearest_idx = np.where(abs(np.exp(result[2])-S)==abs(np.exp(result[2])-S).min())[0]
result[1][nearest_idx, -1]

array([9.18017364])

In [1067]:
result = FD(S, K, r, sigma, T, N, M, scheme="CN")
get_ans(result)

0.9999866666666668


array([9.17929529])

In [1050]:
result[1][:,-1]

array([1.00000000e+00, 9.50082904e-01, 9.00530148e-01, 8.51504602e-01,
       8.03165962e-01, 7.55668863e-01, 7.09161081e-01, 6.63781874e-01,
       6.19660476e-01, 5.76914788e-01, 5.35650262e-01, 4.95959019e-01,
       4.57919188e-01, 4.21594488e-01, 3.87034043e-01, 3.54272422e-01,
       3.23329903e-01, 2.94212933e-01, 2.66914772e-01, 2.41416298e-01,
       2.17686950e-01, 1.95685779e-01, 1.75362580e-01, 1.56659085e-01,
       1.39510185e-01, 1.23845156e-01, 1.09588870e-01, 9.66629689e-02,
       8.49869801e-02, 7.44793628e-02, 6.50584710e-02, 5.66434239e-02,
       4.91548767e-02, 4.25156900e-02, 3.66514952e-02, 3.14911578e-02,
       2.69671420e-02, 2.30157822e-02, 1.95774665e-02, 1.65967412e-02,
       1.40223441e-02, 1.18071746e-02, 9.90821054e-03, 8.28638034e-03,
       6.90639804e-03, 5.73657064e-03, 4.74858440e-03, 3.91727714e-03,
       3.22040258e-03, 2.63839207e-03, 2.15411814e-03, 1.75266369e-03,
       1.42109971e-03, 1.14827403e-03, 9.24612661e-04, 7.41934940e-04,
      

In [997]:
''' Antwoord CN'''
result[2][abs(np.exp(result[2])-S)==abs(np.exp(result[2])-S).min()]
nearest_idx = np.where(abs(np.exp(result[2])-S)==abs(np.exp(result[2])-S).min())[0]
result[1][-1][nearest_idx]

array([0.47340696])

In [989]:
np.exp(result[2])

array([1.00000000e+00, 1.09749877e+00, 1.20450354e+00, 1.32194115e+00,
       1.45082878e+00, 1.59228279e+00, 1.74752840e+00, 1.91791026e+00,
       2.10490414e+00, 2.31012970e+00, 2.53536449e+00, 2.78255940e+00,
       3.05385551e+00, 3.35160265e+00, 3.67837977e+00, 4.03701726e+00,
       4.43062146e+00, 4.86260158e+00, 5.33669923e+00, 5.85702082e+00,
       6.42807312e+00, 7.05480231e+00, 7.74263683e+00, 8.49753436e+00,
       9.32603347e+00, 1.02353102e+01, 1.12332403e+01, 1.23284674e+01,
       1.35304777e+01, 1.48496826e+01, 1.62975083e+01, 1.78864953e+01,
       1.96304065e+01, 2.15443469e+01, 2.36448941e+01, 2.59502421e+01,
       2.84803587e+01, 3.12571585e+01, 3.43046929e+01, 3.76493581e+01,
       4.13201240e+01, 4.53487851e+01, 4.97702356e+01, 5.46227722e+01,
       5.99484250e+01, 6.57933225e+01, 7.22080902e+01, 7.92482898e+01,
       8.69749003e+01, 9.54548457e+01, 1.04761575e+02, 1.14975700e+02,
       1.26185688e+02, 1.38488637e+02, 1.51991108e+02, 1.66810054e+02,
      

In [939]:
result[1][1]

array([ 1.00000000e+00,  1.73074263e-04, -2.88281328e-06,  4.80176098e-08,
       -7.99805824e-10,  1.33219741e-11, -2.21897603e-13,  3.69603976e-15,
       -6.15631252e-17,  1.02542684e-18, -1.70800331e-20,  2.84493751e-22,
       -4.73867316e-24,  7.89297594e-26, -1.31469437e-27,  2.18982206e-29,
       -3.64747943e-31,  6.07542796e-33, -1.01195430e-34,  1.68556274e-36,
       -2.80755936e-38,  4.67641421e-40, -7.78927426e-42,  1.29742345e-43,
       -2.14821572e-45,  7.98665630e-46,  4.53166241e-44,  2.69271193e-42,
        1.59998714e-40,  9.50699115e-39,  5.64897546e-37,  3.35657446e-35,
        1.99444876e-33,  1.18508495e-31,  7.04167668e-30,  4.18410600e-28,
        2.48616115e-26,  1.47725638e-24,  8.77773515e-23,  5.21565757e-21,
        3.09910056e-19,  1.84145990e-17,  1.09418023e-15,  6.50152839e-14,
        3.86315436e-12,  2.29545434e-10,  1.36393997e-08,  8.10441843e-07,
        4.81557835e-05,  2.86137680e-03,  1.70020641e-01,  5.17795599e+00,
        1.60402049e+01,  